    ========= Label =========
    || natural  |        0 ||
    || art      |        1 ||
    || people   |        2 ||
    || food     |        3 ||
    ========= ===== =========

In [13]:
from tensorflow.keras import models
import tensorflow as tf
import numpy as np 
import PIL.Image as pilimg
import os
import shutil

testpath = './test/'
imgpath = './drive/MyDrive/data/Factory/'
# modelpath = './drive/MyDrive/models/Main_Category7(gpu_adam).h5'
modelpath = './drive/MyDrive/models/Main_Category(gpu_adam).h5'

def chkdir(path) :
  if os.path.isdir(path): # 디렉토리가 있다면 그냥 넘기고
    return
  os.mkdir(path)   # 없음 만듬

def movefile(_from, _to):
  # shutil.move(_from, _to) # 어떤파일을, 어디에이동할건지
  shutil.copy2(_from, _to)

def __getPixel(path) :
    img = pilimg.open(path)
    resize_image = img.resize((100, 100))
    pixel = np.array(resize_image)
    return pixel

def getPixels(where = imgpath):
    b = list()
    imglist = os.listdir(where)
    for i in imglist:
        if i == '.DS_Store' : continue
        path = where + i        
        pixel = __getPixel(path)
        if len(pixel.shape) != 3:
          print(i)
          continue
        if pixel.shape[2] != 3: pixel = pixel[..., :3]
        b.append(pixel)
    return np.array(b)

def makeCategory(where = imgpath) :
    with tf.device('/device:GPU:0'):
      model = models.load_model(modelpath)
      img = getPixels(where)
      y_pred = model.predict(img)

      labels = {0 : 'natural', 1: 'art', 2:'people', 3:'food'}
      imglist = os.listdir(where)
      # print(imglist)
      for i in range(len(imglist)):
        image = imglist[i]
        label = labels[y_pred[i].argmax()]
        chkdir(testpath + label) # 디렉토리 없음 만들고
        movefile(where + image, testpath + label + '/' + image)
def reset():
  for i in os.listdir(testpath):
    shutil.rmtree(testpath + i)

In [ ]:
reset()

In [14]:
imgpath2 = './drive/MyDrive/data/please/'
reset()
makeCategory(imgpath2)